In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import squidpy as sq
from scipy.spatial.distance import pdist, squareform
import scipy
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from sklearn import metrics
import tabulate
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import itertools
import umap
from importlib import reload
from scipy import stats
import matplotlib.image as mpimg
import gget

# locals
import utils as ut
reload(ut)

2022-11-15 15:12:40.948201: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 15:12:41.085199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2022-11-15 15:12:41.085228: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-15 15:12:41.111070: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 15:12:42.8254

<module 'utils' from '/home/cstansbu/git_repositories/spatial_transcriptomics/clustering/utils.py'>

In [4]:
clusterPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/scanpy/clusters.csv"
cf = pd.read_parquet(clusterPath)
print(cf.shape)
print(cf['cellType'].unique())
cf.head()

(422574, 6)
['Macrophages' 'B cells' 'NK cells' 'Monocytes' 'Dendritic cells'
 'T cells' 'Stromal cells']


,gene,scores,pvals,clusterId,key,cellType
0,STRADB,1.970233,0.050047,1,ND,Macrophages
1,E2F8,1.969810,0.050184,1,ND,Macrophages
2,EDC3,1.969013,0.050110,1,ND,Macrophages
3,SLC25A20,1.968786,0.050065,1,ND,Macrophages
4,ATP7A,1.968395,0.050189,1,ND,Macrophages


In [20]:
mm = cf[cf['cellType'].isin(['Monocytes', 'Macrophages'])].reset_index(drop=True)

gr = mm.groupby(['cellType', 'key']).agg(
    nGenes = ('gene', 'count'),
    nuniqueGenes = ('gene', 'nunique'),
).reset_index()

gr.head()


# mm[['cell
# Type', 'key']].value_counts().reset_index()



,cellType,key,nGenes,nuniqueGenes
0,Macrophages,HFD14,53394,12970
1,Macrophages,HFD8,55745,13408
2,Macrophages,ND,26873,11509
3,Monocytes,HFD14,10225,8364
4,Monocytes,HFD8,22493,11464


In [9]:
mm['geneCount'] = mm.groupby('gene')['scores'].transform('count')

mm = mm[mm['geneCount'] < 2]

mm.head()

,gene,scores,pvals,clusterId,key,cellType,geneCount
2133,GM13594,0.886328,0.376368,1,ND,Macrophages,1
4854,TRGV2,-0.222568,0.824001,1,ND,Macrophages,1
4917,ELL3,-0.251888,0.801266,1,ND,Macrophages,1
5092,GM11998,-0.343550,0.731372,1,ND,Macrophages,1
5959,LRRN3,-0.755723,0.450256,1,ND,Macrophages,1
